# Churn rate prediction for a telecommunication provider
Milestone: Model Completion

Luwei Wang

Youyu Zhang

In [ ]:
# Import common used Python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

In [ ]:
# Import dataset 
churn_df = pd.read_csv('train.csv')     # Read dataset 
test = pd.read_csv('test.csv')

In [ ]:
churn_df.head()

In [ ]:
churn_df.describe()

In [ ]:
# Check the shape of dataset
churn_df.shape

In [ ]:
# Check the basic information of the dataset 
churn_df.info()

In [ ]:
col_name = churn_df.columns.tolist()
print(col_name)

In [ ]:
churn_df.isnull().sum()   
# No NA values in the dataframe

# sns.heatmap(churn_df.isnull())

In [ ]:
churn_df.area_code.unique()
# Only 3 area_code observed.

In [ ]:
print('Maximum account length is: ', churn_df.account_length.max())
print('Unique account length total number is :', len(churn_df.account_length.unique()))
#np.sort(churn_df.account_length.unique())


In [ ]:
churn_yes = churn_df.loc[churn_df.churn == 'yes',].shape[0]
churn_no = churn_df.loc[churn_df.churn == 'no',].shape[0]
print('Total churn cases number is :', churn_yes , '. It takes proportion of ', round(churn_yes/churn_df.shape[0]*100),'%.')

From the information above, we can see that:
1. This dataset consist of 20 columns. 
2. No Null values observed.

### Data Cleaning

In [ ]:
# Adjust format for variable: area_code
for i in range(len(churn_df)):
    churn_df['area_code'][i] =  int(churn_df['area_code'][i][-3:])

churn_df.head(5)

In [ ]:
'''
churn_df['result'] = 0
churn_df.loc[churn_df.churn == 'yes', 'result'] = 1

churn_df = churn_df.drop(['churn'], axis = 1)
churn_df.rename(columns={"result": "churn"})
'''

churn_df.churn = pd.Series(np.where(churn_df.churn.values == 'yes', 1, 0),
          churn_df.index)
churn_df.international_plan = pd.Series(np.where(churn_df.international_plan.values == 'yes', 1, 0),
          churn_df.index)
churn_df.voice_mail_plan = pd.Series(np.where(churn_df.voice_mail_plan.values == 'yes', 1, 0),
          churn_df.index)
churn_df = churn_df.drop(columns = ['state'])
churn_df.head()

### Plot and Graphs

In [ ]:
churn_df['churn'].value_counts().plot(kind = 'bar')
plt.title("Churn Cases")
plt.ylabel("Number of Cases")


In [ ]:
fig, ax=plt.subplots(figsize=(15,5))
sns.countplot(data = churn_df, x='churn', order=churn_df['churn'].value_counts().index, hue='churn')
plt.xticks(rotation=45)
plt.xlabel('State')
plt.ylabel('Customer Number')
plt.title('Customer Churn Conditions by States')
plt.show()

In [ ]:
fig2, ax=plt.subplots(figsize=(10,5))
sns.countplot(data = churn_df, x='area_code', order=churn_df['area_code'].value_counts().index, hue='churn')
plt.xlabel('Area Code')
plt.ylabel('Customers')
plt.title('Churn Conditions by Area Code')
plt.show()

In [ ]:
churn_df['number_customer_service_calls'].value_counts().plot(kind = 'bar')
plt.title("Statistics of service calls")
plt.ylabel("number")

In [ ]:
plan_yes = churn_df['churn'][churn_df['international_plan'] == 1].value_counts()
plan_no = churn_df['churn'][churn_df['international_plan'] == 0].value_counts()
plan_df = pd.DataFrame({'international plan':plan_yes, 'no international plan':plan_no})

#plan_df.plot(kind = 'bar', stacked = True)
fig3, ax= plt.subplots(figsize=(5,5))
types = ['No','Yes']
ax.bar(types, plan_yes, 0.4, label = 'Internation Plan')  # 0.4 is the width of the bar
ax.bar(types, plan_no , 0.4, bottom = plan_yes, label = 'No International Plan')
ax.set_title("Stat between international plan and churn")
ax.set_xlabel("Churn")
ax.set_ylabel("Number")
ax.legend()
plt.show()


In [ ]:
call_0 = churn_df['number_customer_service_calls'][churn_df['churn'] == 0].value_counts()
call_1 = churn_df['number_customer_service_calls'][churn_df['churn'] == 1].value_counts()
service_df = pd.DataFrame({'churn':call_1, 'no churn':call_0})
service_df.plot(kind = 'bar', stacked = True)
plt.title("Stat between customer service calls and churn")
plt.xlabel("The number of service calls")
plt.ylabel("number")

In [ ]:
var_numbers = [feature for feature in churn_df.columns if churn_df[feature].dtypes != 'O']

for feature in var_numbers:
    sns.distplot(churn_df[feature])
    plt.xlabel(str(feature))
    plt.ylabel('Distribution')
    plt.show()

# Most number features looks like normal distribution

In [ ]:
sns.FacetGrid(churn_df, hue='churn',size = 6.5).map(sns.distplot, 'total_day_minutes').add_legend()
plt.title('Churn and total day minutes')
plt.show()

In [ ]:
churn30 = churn_df.loc[churn_df['total_day_charge']>44]

churn30['churn'].value_counts().plot(kind = 'bar')
plt.title("Churn Cases for total_day_charge > 44")
plt.ylabel("Number of Cases")

#churn_df.total_day_charge

## Correlation Check
The graph below indicates that **total day charge** and **total day minutes** are 100% positive correlated. Same as **total eve minutes** and **total eve charge**, **total night minutes** and **total night charge**, **total intl minutes** and **total intl charge**. This means the charges are linear relative to minutes of calls, which match the common telecommunication service business model. 

Also, multicollinearity exists, and it can lead to inaccurate prediction later. Decision tree and boosted trees can avoid this inaccuracy. Logistic regression and linear regression will lead to skewed results. 

In [ ]:
corr = churn_df.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, vmin=-1, vmax=1)
print(corr)

In [ ]:
corr.columns

One of the methods to solve multicollinearity is PCA. Currently we set the explained variance ratio to 90% (0.9)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, normalize, StandardScaler

# Un-standardized PCA
pcs = PCA(n_components = 0.9)  #Expect explain 90% of variance
pcs.fit(churn_df[corr.columns])
pcsSummary_df = pd.DataFrame({'Standard deviation': np.sqrt(pcs.explained_variance_),
                           'Proportion of variance': pcs.explained_variance_ratio_,
                           'Cumulative proportion': np.cumsum(pcs.explained_variance_ratio_)})
pcsSummary_df= pcsSummary_df.transpose()
pcsSummary_df.columns = ['PC{}'.format(i) for i in range(1, len(pcsSummary_df.columns) + 1)]
pcsSummary_df.round(4)

In [ ]:
pcsComponents_df = pd.DataFrame(pcs.components_.transpose(), columns=pcsSummary_df.columns, 
                                index=corr.columns)
pcsComponents_df   

In [ ]:
# Standardized PCA
norm = StandardScaler()
norm.fit(churn_df[corr.columns])
pcastan = PCA(n_components = 0.9)
pcastan.fit(norm.transform(churn_df[corr.columns]))

pcaSummary_df = pd.DataFrame({'Standard deviation': np.sqrt(pcastan.explained_variance_),
                           'Proportion of variance': pcastan.explained_variance_ratio_,
                           'Cumulative proportion': np.cumsum(pcastan.explained_variance_ratio_)})
pcaSummary_df= pcaSummary_df.transpose()
pcaSummary_df.columns = ['PC{}'.format(i) for i in range(1, len(pcaSummary_df.columns) + 1)]
pcaSummary_df.round(4)

In [ ]:
pcaComponents_df = pd.DataFrame(pcastan.components_.transpose(), columns=pcaSummary_df.columns, 
                                index=corr.columns)
pcaComponents_df   

### Feature Selection with VIF (Variance Inflation Factor) Check
This process will determine if multicollinearity exists and remove the redundent features. 

The criterion is VIF > 0.8. That means we will drop one of the variables from variable pairs with VIF > 0.8. 

In [ ]:
from sklearn.metrics import r2_score

vif = pd.DataFrame(columns = ['var1', 'var2', 'R2'])
for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        if i != j:
            r2value = r2_score(churn_df[corr.columns[i]],churn_df[corr.columns[j]])
            vifvalue = 1/(1-r2value)
            if vifvalue >= 0.8:
                vif = vif.append({'var1' : corr.columns[i], 'var2' : corr.columns[j], 'R2' : vifvalue}, 
                ignore_index = True)

vif

In [ ]:
#Delete features listed in vif.var1
df = churn_df
df = df.drop(vif.var1.unique(),axis = 1)

df

### Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Split dataset into train and test
df2 = df.drop(['churn'], axis = 1)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(df2,df.churn,test_size=0.2)

# Add model and predict
score = []
acc = []
cross = []
for i in range(20):
    decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini', max_depth=i+1)
    decision_tree.fit(Xtrain, Ytrain)
    Y_pred = decision_tree.predict(Xtest)
    
    acc_decision_tree = round(decision_tree.score(Xtest,Y_pred,)*100, 2)
    acc.append(acc_decision_tree)

    result = pd.DataFrame()
    result['Test'] = Ytest
    result['Prediction'] = Y_pred
    result['correct'] = pd.Series(np.where(result.Test.values == result.Prediction.values, 1, 0),
            result.index)
    modelscore = round(result.correct.sum()/result.shape[0],2)
    score.append(modelscore)

    cross_value = cross_val_score(decision_tree, Xtrain, Ytrain, cv = 5)
    cross.append(cross_value)

cross_validation = pd.DataFrame(cross)
cross_validation['avg'] = cross_validation.mean(axis = 1)

print(score)
print(cross_validation)

# Thus max_depth = 5 would be a better choice.


In [ ]:
rs = np.array(cross_validation['avg'])
plt.figure(figsize = (20,5))
plt.plot(range(20),rs,'o-',c = 'black', label = 'Decision_tree')
x=np.arange(0,20,1)
plt.xticks(x)
plt.xlabel('Max_depth')
plt.ylabel('cross_validation_score')

### Use XGBoost Classifier as a Method

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
# Split dataset into train and test
df2 = churn_df.drop(['churn'], axis = 1)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(df2,df.churn,test_size=0.2)
Xtest['area_code'] = lbl.fit_transform(Xtest['area_code'].astype(str))
Xtrain['area_code'] = lbl.fit_transform(Xtrain['area_code'].astype(str))

# Add model and predict
score_xgb = []
acc = []
cross_xgb = []
for i in range(20):
    xgb_model = xgb.XGBClassifier(eta=0.3, max_depth=i+1, colsample_bytree=0.5, scale_pos_weight=1.1, booster='gbtree', random_state= 42)
    xgb_pred = xgb_model.fit(Xtrain._get_numeric_data(), np.ravel(Ytrain, order='C')).predict(Xtest._get_numeric_data())    
    
    acc_xgb = round(xgb_model.score(Xtest,xgb_pred,)*100, 2)
    acc.append(acc_xgb)

    result = pd.DataFrame()
    result['Test'] = Ytest
    result['Prediction'] = xgb_pred
    result['correct'] = pd.Series(np.where(result.Test.values == result.Prediction.values, 1, 0),
            result.index)
    modelscore = round(result.correct.sum()/result.shape[0],2)
    score_xgb.append(modelscore)

    cross_value = cross_val_score(xgb_model, Xtrain, Ytrain, cv = 5)
    cross_xgb.append(cross_value)

cross_validation_xgb = pd.DataFrame(cross_xgb)
cross_validation_xgb['avg'] = cross_validation_xgb.mean(axis = 1)

print(score_xgb)
print(cross_validation_xgb)
# Thus max_depth = 4 would be a better choice.
# Adjusting eta doesn't make big difference.

In [ ]:
rs = np.array(cross_validation_xgb['avg'])
plt.figure(figsize = (20,5))
plt.plot(range(20),rs,'o-',c = 'black', label = 'XGBoost')
x=np.arange(0,20,1)
plt.xticks(x)
plt.xlabel('n_estimators')
plt.ylabel('cross_validation_score_xgb')

### Neuron Network Trial

In [ ]:
from sklearn.neural_network import MLPClassifier

# Add model and predict
score_mlp = []
acc_mlp = []
cross_mlp = []
for i in range(5):
        mlp = MLPClassifier(hidden_layer_sizes=(2,2,2,3,i+1), activation='identity', solver='lbfgs', random_state=42)
        mlp.fit(Xtrain, Ytrain)
        Y_pred = mlp.predict(Xtest)

        acc = round(mlp.score(Xtest,Y_pred,)*100, 2)
        acc_mlp.append(acc)

        result = pd.DataFrame()
        result['Test'] = Ytest
        result['Prediction'] = Y_pred
        result['correct'] = pd.Series(np.where(result.Test.values == result.Prediction.values, 1, 0),
                result.index)
        modelscore = round(result.correct.sum()/result.shape[0],2)
        score_mlp.append(modelscore)

        cross_value = cross_val_score(mlp, Xtrain, Ytrain, cv = 5)
        cross_mlp.append(cross_value)

cross_validation_mlp = pd.DataFrame(cross_mlp)
cross_validation_mlp['avg'] = cross_validation_mlp.mean(axis = 1)

print(score_mlp)
print(cross_validation_mlp)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

score_knn = []
acc_knn = []
cross_knn = []

for i in range(10):
        neigh = KNeighborsClassifier(n_neighbors=i+1)
        neigh.fit(Xtrain, Ytrain)
        Y_pred = neigh.predict(Xtest)

        result = pd.DataFrame()
        result['Test'] = Ytest
        result['Prediction'] = Y_pred
        result['correct'] = pd.Series(np.where(result.Test.values == result.Prediction.values, 1, 0),
                result.index)
        modelscore = round(result.correct.sum()/result.shape[0],2)
        score_knn.append(modelscore)

        cross_value = cross_val_score(neigh, Xtrain, Ytrain, cv = 5)
        cross_knn.append(cross_value)

cross_validation_knn = pd.DataFrame(cross_knn)
cross_validation_knn['avg'] = cross_validation_knn.mean(axis = 1)

print(score_knn)
print(cross_validation_knn)

#### Prediction
Use the models to predict the test.csv, then upload to Kaggle for test. 

In [ ]:
# Adjust format for variable: area_code
test.area_code = lbl.fit_transform(test.area_code.astype(str))

#test.churn = pd.Series(np.where(test.churn.values == 'yes', 1, 0), test.index)
test.international_plan = pd.Series(np.where(test.international_plan.values == 'yes', 1, 0),
          test.index)
test.voice_mail_plan = pd.Series(np.where(test.voice_mail_plan.values == 'yes', 1, 0),
          test.index)
id = test['id']
tests = test.drop(['id','state',], axis = 1)

print(test.shape)
print(test.columns)
print(Xtrain.shape)
print(Xtrain.columns)
print(tests.shape)


Use Decision Tree for prediction

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini', max_depth=5)
decision_tree.fit(Xtrain, Ytrain)
Y_test = decision_tree.predict(tests)

submit = pd.DataFrame({'id':id, 'churn':Y_test})
submit.churn.replace([0,1],['no','yes'], inplace=True)
submit.head()
submit.to_csv('churn_submit.csv',index=False)

In [ ]:
from sklearn import tree
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(30,15))
_ = tree.plot_tree(decision_tree, fontsize=8,
                   feature_names=Xtrain.columns,  
                   class_names='target',
                   filled=True)

Use XGBoost Classifier for prediction


In [ ]:
xgboost = xgb.XGBClassifier(eta=0.3, max_depth=4, colsample_bytree=0.5, scale_pos_weight=1.1, booster='gbtree', random_state= 42)
xgboost.fit(Xtrain, Ytrain)
Y_test_xgb = xgboost.predict(tests)

submit_xgb = pd.DataFrame({'id':id, 'churn':Y_test_xgb})
submit_xgb.churn.replace([0,1],['no','yes'], inplace=True)
submit_xgb.to_csv('churn_submitxgboost.csv',index=False)

In [ ]:
# plot decision tree
import graphviz
from numpy import loadtxt
from xgboost import plot_tree

# plot single tree
xgb.plot_tree(xgboost, num_trees=0, rankdir='LR')
fig = plt.gcf()
fig.set_size_inches(150, 100)

Use Neuron Network for prediction

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(2,2,2,3,2), activation='identity', solver='lbfgs', random_state=42)
mlp.fit(Xtrain, Ytrain)
Y_pred_mlp = mlp.predict(tests)

submit_mlp = pd.DataFrame({'id':id, 'churn':Y_pred_mlp})
submit_mlp.churn.replace([0,1],['no','yes'], inplace=True)
submit_mlp.to_csv('churn_submitmlp.csv',index=False)

Use KNN for prediction

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(Xtrain, Ytrain)
Y_pred_knn = neigh.predict(tests)

submit_knn = pd.DataFrame({'id':id, 'churn':Y_pred_knn})
submit_knn.churn.replace([0,1],['no','yes'], inplace=True)
submit_knn.to_csv('churn_submitknn.csv',index=False)

Timing Calculation

In [ ]:
import time
results = []
max_depth = [1, 2, 3, 4,5,6,7,8,9,10]

for n in max_depth:
    start = time.time()
    decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini', max_depth=n)
    decision_tree.fit(Xtrain, Ytrain)
    elapsed = time.time() - start
    print(n,round(elapsed,3))
    results.append(elapsed)
results_df=pd.DataFrame(results)
results_df.mean()

In [ ]:

results = []
max_depth = [1, 2, 3, 4,5,6,7,8,9,10]
for n in max_depth:
    start = time.time()
    xgboost = xgb.XGBClassifier(eta=0.3,max_depth=n)
    xgboost.fit(Xtrain, Ytrain)
    elapsed = time.time() - start
    print(n,round(elapsed,3))
    results.append(elapsed)
results_df=pd.DataFrame(results)
results_df.mean()

In [ ]:
results = []
hls = [(5),(5,5),(5,5,5),(5,5,5,5),(5,5,5,5,5),(2),(2,2),(2,2,2),(2,2,2,2),(2,2,2,2,2)]

for n in hls:
    start = time.time()
    mlp = MLPClassifier(hidden_layer_sizes=n, activation='identity', solver='lbfgs', random_state=42)
    mlp.fit(Xtrain, Ytrain)
    elapsed = time.time() - start
    print(n,round(elapsed,3))
    results.append(elapsed)
results_df=pd.DataFrame(results)
results_df.mean()

In [ ]:
results = []

for n in range(10):
    start = time.time()
    neigh = KNeighborsClassifier(n_neighbors=i+1)
    neigh.fit(Xtrain, Ytrain)
    elapsed = time.time() - start
    print(n,round(elapsed,3))
    results.append(elapsed)
results_df=pd.DataFrame(results)
results_df.mean()

In [ ]:
from dmba import classificationSummary
classificationSummary(Ytest, decision_tree.predict(Xtest))

In [ ]:
classificationSummary(Ytest, xgb_model.predict(Xtest))

In [ ]:
classificationSummary(Ytest, mlp.predict(Xtest))

In [ ]:
classificationSummary(Ytest, neigh.predict(Xtest))